In [1]:
from surprise import Dataset
from surprise import SVD
from surprise import model_selection
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

## Automatic cross-validation

In [2]:
data = Dataset.load_builtin('ml-100k')

In [3]:
data

In [4]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9371  0.9342  0.9298  0.9396  0.9418  0.9365  0.0042  
MAE (testset)     0.7399  0.7371  0.7337  0.7384  0.7406  0.7379  0.0024  
Fit time          3.31    3.29    3.32    3.32    3.32    3.31    0.01    
Test time         0.11    0.15    0.11    0.10    0.14    0.12    0.02    


{'test_rmse': array([0.93713818, 0.9342457 , 0.92984112, 0.93959251, 0.94181479]),
 'test_mae': array([0.73985849, 0.73705624, 0.73374078, 0.73844572, 0.74055128]),
 'fit_time': (3.3071305751800537,
  3.2941954135894775,
  3.3192172050476074,
  3.322118043899536,
  3.3201229572296143),
 'test_time': (0.10870885848999023,
  0.15056800842285156,
  0.10571694374084473,
  0.10175132751464844,
  0.13962745666503906)}

## Train-test split and the fit() method

In [5]:
model = SVD()
trainset, testset = train_test_split(data, test_size=0.25, random_state=10, shuffle=True)
model.fit(trainset)
pred = model.test(testset)
accuracy.rmse(pred)

RMSE: 0.9321


0.9320816817244147

## Use a custom dataset

In [7]:
from surprise import BaselineOnly
from surprise import Reader
import os

file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(file_path, reader=reader)
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9463  0.9436  0.9436  0.9449  0.9403  0.9437  0.0020  
MAE (testset)     0.7522  0.7485  0.7459  0.7477  0.7463  0.7481  0.0022  
Fit time          0.15    0.16    0.16    0.16    0.15    0.16    0.01    
Test time         0.08    0.07    0.07    0.14    0.07    0.09    0.03    


{'test_rmse': array([0.94628852, 0.94364818, 0.9435601 , 0.94489797, 0.9402699 ]),
 'test_mae': array([0.7522021 , 0.74851653, 0.74593583, 0.74772131, 0.74631545]),
 'fit_time': (0.14760494232177734,
  0.1576073169708252,
  0.1625657081604004,
  0.1605982780456543,
  0.14663004875183105),
 'test_time': (0.0758211612701416,
  0.07277631759643555,
  0.07380247116088867,
  0.14259076118469238,
  0.06879377365112305)}

### load a dataset from a pandas dataframe

In [1]:
import pandas as pd
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

C:\Users\Houlai\Anaconda3\envs\TF2_GPU_PY36\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Houlai\Anaconda3\envs\TF2_GPU_PY36\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
ratings_dict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 'user_foo'],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratings_dict)

In [3]:
df[['userID', 'itemID', 'rating']]

,userID,itemID,rating
0,9,1,3
1,32,1,2
2,2,1,4
3,45,2,3
4,user_foo,2,1


In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)
model = NormalPredictor()
cross_validate(model, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8481  1.0478  0.3203  0.2766  1.9844  1.0954  0.7253  
MAE (testset)     1.8481  1.0478  0.3203  0.2766  1.9844  1.0954  0.7253  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.84805728, 1.04777679, 0.32027995, 0.27660912, 1.98443767]),
 'test_mae': array([1.84805728, 1.04777679, 0.32027995, 0.27660912, 1.98443767]),
 'fit_time': (0.0, 0.0, 0.000997304916381836, 0.0, 0.0),
 'test_time': (0.000997304916381836, 0.0, 0.0, 0.0, 0.0)}

## Use cross-validation iterators
For cross-validation, we can use the cross_validate() function that does all the hard work for us. But for a better control, we can also instanciate a cross-validation iterator, and make predictions over each split using the split() method of the iterator, and the test() method of the algorithm. Here is an example where we use a classical K-fold cross-validation procedure with 5 splits:

In [5]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import Dataset

In [6]:
data = Dataset.load_builtin("ml-100k")
model = SVD()
kf = KFold(n_splits=5)
rmse = []
for trainset, testset in kf.split(data):
    model.fit(trainset)
    pred = model.test(testset)
    rmse.append(accuracy.rmse(pred))
rmse

RMSE: 0.9381
RMSE: 0.9235
RMSE: 0.9416
RMSE: 0.9368
RMSE: 0.9336


[0.938104942837033,
 0.9235084141941013,
 0.9415713497791371,
 0.9368151342405137,
 0.9336001494262185]

In [11]:
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin('ml-100k')
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005, .01],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)
# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.956063972717839
{'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.4}


In [12]:
gs.best_estimator

{'rmse': <surprise.prediction_algorithms.matrix_factorization.SVD at 0x27f4c992470>,
 'mae': <surprise.prediction_algorithms.matrix_factorization.SVD at 0x27f59868f98>}